# Retriever and Chain with Langchain

In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [20]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(r"C:\Fajri\llm-langchain\rag\attention.pdf")
text_docs = loader.load()
text_docs[:2]

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'source': 'C:\\Fajri\\llm-langchain\\rag\\attention.pdf', 'file_path': 'C:\\Fajri\\llm-langchain\\rag\\attention.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'trapped': '', 'modDate': 'D:20230803000729Z', 'creationDate': 'D:20230803000729Z', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.tor

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20
)

documents = text_splitter.split_documents(text_docs)


In [22]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents[:20], OpenAIEmbeddings())  # Limit to first 20 documents for demo purposes

In [23]:
query = "An attention function can be described as mapping a query"
result = db.similarity_search(query, k=3)
result[0].page_content


'Instead of performing a single attention function with dmodel-dimensional keys, values and queries,\nwe found it beneficial to linearly project the queries, keys and values h times with different, learned\nlinear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of\nqueries, keys and values we then perform the attention function in parallel, yielding dv-dimensional\n4To illustrate why the dot products get large, assume that the components of q and k are independent random\nvariables with mean 0 and variance 1. Then their dot product, q · k = Pdk\ni=1 qiki, has mean 0 and variance dk.\n4'

In [24]:
from langchain_openai import ChatOpenAI

gpt = ChatOpenAI(model="gpt-4")

In [25]:
# Desogm ChatPrompt Template
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context.
    Think step by step before providing a detailded answer.
    I will tip you $1000 if the user finds the answer helpful.
    <context>
    {context}
    </context>
    Question: {input}
    """
)

In [26]:
# Stuff Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

documents_chain = create_stuff_documents_chain(
    llm=gpt,
    prompt=prompt
)

In [27]:
# Retriiever
"""
Retrievers: A retriever is an interface that returns documents given
an unstructured query. It is more general than a vector store.
A retriever does not need to be able to store documents, only to 
return (or retrieve) them. Vector stores can be used as the backbone
of a retriever, but there are other types of retrievers as well. 
https://python.langchain.com/docs/modules/data_connection/retrievers/ 
"""
    
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000290C01E45D0>, search_kwargs={})

In [28]:
# Retrieval Chain
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever, documents_chain)

In [33]:
response = retrieval_chain.invoke({"input":"What is Pusat Monitoring Bendungan"})


In [34]:
response

{'input': 'What is Pusat Monitoring Bendungan',
 'context': [Document(id='0f3089b8-0696-4f8d-81ee-16b5b0a8049c', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'source': 'C:\\Fajri\\llm-langchain\\rag\\attention.pdf', 'file_path': 'C:\\Fajri\\llm-langchain\\rag\\attention.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'trapped': '', 'modDate': 'D:20230803000729Z', 'creationDate': 'D:20230803000729Z', 'page': 0}, page_content='has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head\nattention and the parameter-free position representation and became the other person involved in nearly every\ndetail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and\ntensor2tensor. Llion also experimented with novel model variants,

In [35]:
response["answer"]

'The text does not provide information on what Pusat Monitoring Bendungan is.'